# STIX-Shifter CLI Quick Lab

## Overview

STIX (Structured Threat Information eXpression) is a JSON structure used to share cybersecurity threat intelligence. STIX-shifter is an open-source python library that is part of the Open Cybersecurity Alliance. It allows data repositories to be queried using STIX patterning and return the results as STIX cyber observable objects. This lab will allow users to test out the various stix-shifter CLI commands.

### STIX Patterning

A [STIX pattern](http://docs.oasis-open.org/cti/stix/v2.0/cs01/part5-stix-patterning/stix-v2.0-cs01-part5-stix-patterning.html) is used to query [cyber observable objects](https://docs.oasis-open.org/cti/stix/v2.0/stix-v2.0-part4-cyber-observable-objects.html). STIX patterns take the format of:

`[<OBJECT>:<PROPERTY> = 'some value' AND <OBJECT>:<PROPERTY> IN (value_1, value_2)] OR [<OBJECT>:<PROPERTY> = 'some value']`

The `[ ]` represents one observation. A pattern can have multiple observations joined by the AND or OR observation operators. An observation can be thought of as one instance or row of data. Within the observation is one or more comparison expressions that looks for a value associated to a cyber observable STIX object and its property. This is a sample pattern with one observation containing an comparison operation for an IP lookup: `[ipv4-addr:value = '1.2.3.4']`. The STIX object in this case is `ipv4-addr` and the property on that object is `value`.

### STIX Observed Data

STIX-shifter returns a `bundle` of STIX `observed-data` objects. The bundle is a container object to hold the results. Below is a sample bundle containing one identity object (representing the data source) and one observed-data object:

```json
{
    "type": "bundle",
    "id": "bundle--57d455df-105d-4722-8277-e569110e82ed",
    "objects": [
        {
            "type": "identity",
            "id": "identity--f431f809-377b-45e0-aa1c-6a4751cae5ff",
            "name": "QRadar",
            "identity_class": "system"
        },
        {
            "id": "observed-data--4db61897-4725-483b-9e68-2874e48650c5",
            "type": "observed-data",
            "created_by_ref": "identity--f431f809-377b-45e0-aa1c-6a4751cae5ff",
            "created": "2022-04-28T14:16:41.544Z",
            "modified": "2022-04-28T14:16:41.544Z",
            "objects": {
                "0": {
                    "type": "x-oca-event",
                    "action": "Logon Failure - Unknown user name or bad password",
                    "outcome": "Host Login Failed",
                    "category": [
                        "Authentication"
                    ],
                    "provider": "Microsoft Windows Security Event Log",
                    "agent": "WindowsAuthServer @ microsoft.windows.test.com",
                    "created": "2021-06-28T19:35:58.000Z",
                    "network_ref": "2",
                    "user_ref": "4",
                    "url_ref": "7",
                    "file_ref": "8"
                },
                "1": {
                    "type": "ipv4-addr",
                    "value": "109.0.216.203"
                },
                "2": {
                    "type": "network-traffic",
                    "src_ref": "1",
                    "src_port": 3000,
                    "dst_ref": "3",
                    "dst_port": 1000,
                    "protocols": [
                        "TCP"
                    ]
                },
                "3": {
                    "type": "ipv4-addr",
                    "value": "192.168.1.11"
                },
                "4": {
                    "type": "user-account",
                    "user_id": "bill_holland"
                },
                "5": {
                    "type": "ipv4-addr",
                    "value": "0.0.0.0"
                },
                "6": {
                    "type": "artifact",
                    "payload_bin": "PDEzPk1hciAyMSAwMTo0Mjo1MCBtaWNyb3NvZnQud2luZG93cy50ZXN0LmNvbQ==",
                    "mime_type": "text/plain"
                },
                "7": {
                    "type": "url",
                    "value": "www.example.com"
                },
                "8": {
                    "type": "file",
                    "name": "myfile.exe",
                    "hashes": {
                        "SHA-256": "86c5ceb27e1bf441130299c0209e5f35b88089f62c06b2b09d65772274f12057"
                    },
                    "parent_directory_ref": "9"
                },
                "9": {
                    "type": "directory",
                    "path": "C://filepath"
                }
            },
            "first_observed": "2021-06-28T19:35:58.652Z",
            "last_observed": "2021-06-28T19:36:58.652Z",
            "number_observed": 31
        }
    ],
    "spec_version": "2.0"
}
```

Each observed-data object contains a numbered set of cyber-observable objects. The properties on the cyber-observable object store the data returned from the data source. See the [STIX 2.0 standard](https://docs.oasis-open.org/cti/stix/v2.0/stix-v2.0-part4-cyber-observable-objects.html) for more on cyber observable objects.

## Setup

### Prerequisites

* Python 3
* pip
* venv
* Ability to run bash commands

### 1. Open a terminal and install a Python Virtual Environment

`python3 -m venv labenv`

`source labenv/bin/activate`

### 2. Install jupyter notebook

`pip install notebook`

### 3. Run jupyter notebook

`jupyter notebook`

### All remaining steps take place in the jupyter notebook

### 4. Install the required stix-shifter libraries

This installs the core stix-shifter and utils library along with the STIX-bundle and QRadar connectors.

In [25]:
%%bash
pip install \
stix-shifter \
stix-shifter-utils \
stix-shifter-modules-stix_bundle \
stix-shifter-modules-qradar \
stix-shifter-modules-mysql \
stix-shifter-modules-reaqta

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


Looking in indexes: https://pypi.org/simple, https://pypi.org/simple


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


# Lab Exercise 1: Using CLI tools with the STIX-Bundle connector

## Examine the STIX Bundle
This is a bundle of STIX observed-data objects containing sanitized data from a CrowdStrike instance.

https://raw.githubusercontent.com/opencybersecurityalliance/stix-shifter/develop/data/cybox/crowdstrike/crowdstrike_detections_20210723.json

The stix_bundle connector will query the sample STIX bundle and return a subset of data based on the query pattern.

Note the bundle of observed-data objects that are returned. Each of these objects contains a numbered set of cyber observable objects (`url`, `network-traffic`, `ipv4-addr`…) which contain the data from the target data source. Given the above CLI example, the `ipv4-addr` object should contain a value property with **12.111.222.0**

## STIX-Shifter Transmission CLI commands
The transmission commands use the data source APIs to send a query, check the status, fetch the results, and ping the connection.

### Set environment variables to be used in the CLI

In [19]:
%env BUNDLE_URL https://raw.githubusercontent.com/opencybersecurityalliance/stix-shifter/develop/data/cybox/crowdstrike/crowdstrike_detections_20210723.json
    


env: BUNDLE_URL=https://raw.githubusercontent.com/opencybersecurityalliance/stix-shifter/develop/data/cybox/crowdstrike/crowdstrike_detections_20210723.json


In [20]:
%env IDENTITY_OBJECT \
{ \
 "type":"identity", \
 "id":"identity--f431f809-377b-45e0-aa1c-6a4751cae5ff", \
 "name":"STIX Bundle", \
 "identity_class":"system", \
 "created": "2022-04-07T20:35:41.042Z", \
 "modified": "2022-04-07T20:35:41.042Z" \
}   

env: IDENTITY_OBJECT={   "type":"identity",   "id":"identity--f431f809-377b-45e0-aa1c-6a4751cae5ff",   "name":"STIX Demo",   "identity_class":"system",   "created": "2022-04-07T20:35:41.042Z",   "modified": "2022-04-07T20:35:41.042Z"  }


In [21]:
%env BUNDLE_AUTH {"auth": {}}

env: BUNDLE_AUTH={"auth": {}}


### Run the ping command
This command checks that the data source can be reached by the stix-shifter connector.

In [29]:
%%bash
stix-shifter transmit stix_bundle '{"url": "'"$BUNDLE_URL"'"}' "$BUNDLE_AUTH" ping

{
    "success": true
}


### Run the query command
This command sends the native query to the data source.

In [17]:
%%bash
stix-shifter transmit stix_bundle '{"url": "'"$BUNDLE_URL"'"}' "$BUNDLE_AUTH" query "[ipv4-addr:value = '192.168.0.8']"

{
    "success": true,
    "search_id": "[ipv4-addr:value = '192.168.0.8']"
}


### Run the status command
This command checks the status of the query.

In [16]:
%%bash
stix-shifter transmit stix_bundle '{"url": "'"$BUNDLE_URL"'"}' "$BUNDLE_AUTH" status "[ipv4-addr:value = '192.168.0.8']"

{
    "success": true,
    "status": "COMPLETED",
    "progress": 100
}


### Run the results command
This command fetches the query results

In [15]:
%%bash
stix-shifter transmit stix_bundle '{"url": "'"$BUNDLE_URL"'"}' "$BUNDLE_AUTH" results "[ipv4-addr:value = '192.168.6.242']" 0 2

{
    "success": true,
    "data": [
        {
            "id": "observed-data--05baca02-9154-45e3-a41d-f10366ad8dc0",
            "type": "observed-data",
            "created_by_ref": "identity--fa188421-a904-4e95-a3a4-309a558b9295",
            "created": "2021-07-23T14:24:43.723Z",
            "modified": "2021-07-23T14:24:43.723Z",
            "objects": {
                "0": {
                    "type": "x-oca-event",
                    "created": "2021-06-17T11:36:21Z",
                    "process_ref": "2",
                    "action": "CustomIOAWinHigh",
                    "outcome": "A process triggered a high severity custom rule.",
                    "severity": 70,
                    "parent_process_ref": "7",
                    "host_ref": "9",
                    "provider": "CrowdStrike"
                },
                "1": {
                    "type": "file",
                    "name": "powershell.exe",
                    "parent_directory_ref": "3",
  

                        "8",
                        "10"
                    ],
                    "hostname": "WIN10-1S",
                    "mac_refs": [
                        "11"
                    ],
                    "os_version": "Windows 10",
                    "os_platform": "Windows"
                },
                "10": {
                    "type": "ipv4-addr",
                    "value": "192.168.6.242"
                },
                "11": {
                    "type": "mac-addr",
                    "value": "00:0a:11:f1:00:0x"
                }
            },
            "first_observed": "2021-05-10T18:45:39Z",
            "last_observed": "2021-05-10T18:45:39Z",
            "number_observed": 1
        }
    ]
}


## STIX-Shifter Execute CLI command

### Run the execute command
The execute command runs through the entire stix-shifter flow:

* Translates a STIX pattern into a native data source query
* Sends the query to the data source via the data source APIs
* Checks the status of the query via the data source APIs
* Fetches the query results via the APIs and, if needed, converts them to JSON
* Translates the JSON results into STIX objects

In [22]:
%%bash
stix-shifter execute stix_bundle stix_bundle "$IDENTITY_OBJECT" '{"url": "'"$BUNDLE_URL"'"}' \
"$BUNDLE_AUTH" "[ipv4-addr:value = '12.111.222.0']"

 2022-08-23 14:45:15,272 stix_shifter.scripts.stix_shifter INFO     Translated Queries: 
{
    "queries": [
        "[ipv4-addr:value = '12.111.222.0']"
    ]
}
 2022-08-23 14:45:15,731 stix_shifter.scripts.stix_shifter INFO     STIX Results (written to stdout):



{
    "type": "bundle",
    "id": "bundle--7bac91c6-368d-40f4-8659-97f2f268e844",
    "objects": [
        {
            "type": "identity",
            "id": "identity--f431f809-377b-45e0-aa1c-6a4751cae5ff",
            "name": "STIX Demo",
            "identity_class": "system",
            "created": "2022-04-07T20:35:41.042Z",
            "modified": "2022-04-07T20:35:41.042Z"
        },
        {
            "id": "observed-data--05baca02-9154-45e3-a41d-f10366ad8dc0",
            "type": "observed-data",
            "created_by_ref": "identity--f431f809-377b-45e0-aa1c-6a4751cae5ff",
            "created": "2021-07-23T14:24:43.723Z",
            "modified": "2021-07-23T14:24:43.723Z",
            "objects": {
                "0": {
                    "type": "x-oca-event",
                    "created": "2021-06-17T11:36:21Z",
                    "process_ref": "2",
                    "action": "CustomIOAWinHigh",
                    "outcome": "A process triggered a high severit

                    "pid": 22456494518
                },
                "8": {
                    "type": "ipv4-addr",
                    "value": "12.111.222.0"
                },
                "9": {
                    "type": "x-oca-asset",
                    "ip_refs": [
                        "8",
                        "10"
                    ],
                    "hostname": "WIN10-1S",
                    "mac_refs": [
                        "11"
                    ],
                    "os_version": "Windows 10",
                    "os_platform": "Windows"
                },
                "10": {
                    "type": "ipv4-addr",
                    "value": "192.168.6.242"
                },
                "11": {
                    "type": "mac-addr",
                    "value": "00:0a:11:f1:00:0x"
                }
            },
            "first_observed": "2021-05-10T18:45:39Z",
            "last_observed": "2021-05-10T18:45:39Z",
           

                    "technique_id": "CST0018",
                    "confidence": 80,
                    "detection_id": "ldt:0a3e7f1d15874c09ad37dae363774e50:21477405166",
                    "agent_local_time": "2021-05-10T16:53:04.627Z",
                    "agent_version": "6.22.13607.0",
                    "ioc_value": "VMware, Inc.",
                    "first_seen": "2021-05-10T16:45:12Z",
                    "last_seen": "2021-05-10T18:21:42Z",
                    "platform_id": "0"
                },
                "5": {
                    "type": "user-account",
                    "account_login": "admin",
                    "user_id": "A-2-3-4-333-444-222-1001"
                },
                "6": {
                    "type": "file",
                    "hashes": {
                        "SHA-256": "f95b7ba752c6452da9d83f84ca7307ae079d220718bcb2babf145903bac894dd",
                        "MD5": "b5a6d2fb3f4521c37d613de52ab3467d"
                    }
            

                    "type": "process",
                    "binary_ref": "1",
                    "name": "powershell.exe",
                    "command_line": "powershell -ep bypass -c \"(0..61)|%{$s+=[char][byte]('0x'+'4765742D576D694F626A6563742057696E33325F536861646F77636F7079207C20466F72456163682D4F626A656374207B245F2E44656C65746528293B7D20'.Substring(2*$_,2))};iex $s\"",
                    "creator_user_ref": "5",
                    "pid": 22475614444,
                    "parent_ref": "7"
                },
                "3": {
                    "type": "directory",
                    "path": "\\Device\\HarddiskVolume4\\Windows\\System32\\WindowsPowerShell\\v1.0\\powershell.exe"
                },
                "4": {
                    "type": "x-crowdstrike",
                    "scenario": "data_loss",
                    "tactic": "Impact",
                    "tactic_id": "TA0040",
                    "technique": "Data Encrypted for Impact",
                    "

                    "parent_process_ref": "7",
                    "host_ref": "9",
                    "provider": "CrowdStrike"
                },
                "1": {
                    "type": "file",
                    "name": "powershell.exe",
                    "parent_directory_ref": "3",
                    "hashes": {
                        "SHA-256": "de96a6e69944335375dc1ac238336066889d9ffc7d73628ef4fe1b1b160ab32c",
                        "MD5": "7353f60b1739074eb17c5f4dddefe239"
                    }
                },
                "2": {
                    "type": "process",
                    "binary_ref": "1",
                    "name": "powershell.exe",
                    "command_line": "powershell.exe  Invoke-WebRequest -Uri pastebin.com",
                    "creator_user_ref": "5",
                    "pid": 22824166752,
                    "parent_ref": "7"
                },
                "3": {
                    "type": "directory",
           